In [ ]:
BRANCH = 'main'

"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""

In [ ]:
import os
# either provide a path to local NeMo repository with NeMo already installed or git clone

# option #1: local path to NeMo repo with NeMo already installed
NEMO_DIR_PATH = "NeMo"

# option #2: download NeMo repo
if 'google.colab' in str(get_ipython()) or not os.path.exists(NEMO_DIR_PATH):
  ! git clone -b $BRANCH https://github.com/NVIDIA/NeMo
  ! cd NeMo
  ! python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
import json
import os
import wget

from IPython.display import Audio
import numpy as np
import scipy.io.wavfile as wav

! pip install pandas
! pip install plotly
from plotly import graph_objects as go

# 1. Introduction
End-to-end Automatic Speech Recognition (ASR) systems surpassed traditional systems in performance but require large amounts of labeled data for training. 

This tutorial will show how to use a pre-trained with Connectionist Temporal Classification (CTC) ASR model, such as [QuartzNet Model](https://arxiv.org/abs/1910.10261) or [Citrinet](https://arxiv.org/abs/2104.01721) to split long audio files and the corresponding transcripts into shorter fragments that are suitable for an ASR model training. 

We're going to use [ctc-segmentation](https://github.com/lumaku/ctc-segmentation) Python package based on the algorithm described in [CTC-Segmentation of Large Corpora for German End-to-end Speech Recognition](https://arxiv.org/pdf/2007.09127.pdf).

In [ ]:
requirements = f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/tools/ctc_segmentation/requirements.txt'
wget.download(requirements)
! pip install -r requirements.txt
! apt-get install -y ffmpeg
! apt-get install -y libsox-fmt-mp3 

`TOOLS_DIR` contains scripts that we are going to need during the next steps, all necessary scripts could be found [here](https://github.com/NVIDIA/NeMo/tree/main/tools/ctc_segmentation/scripts).

In [ ]:
if 'google.colab' in str(get_ipython()):
  NEMO_DIR_PATH = "/content/NeMo"
elif not os.path.exists(NEMO_DIR_PATH):
  NEMO_DIR_PATH = "NeMo"
    
TOOLS_DIR = f'{NEMO_DIR_PATH}/tools/ctc_segmentation/scripts'
print(TOOLS_DIR)
! ls -l $TOOLS_DIR

# 2. Data Download
First, let's download audio and text data (data source: [https://librivox.org/](https://librivox.org/) and [http://www.gutenberg.org](http://www.gutenberg.org).

In [ ]:
## create data directory and download an audio file
WORK_DIR = 'WORK_DIR'
DATA_DIR = WORK_DIR + '/DATA'
os.makedirs(DATA_DIR, exist_ok=True)

print('downloading audio samples...')
wget.download("https://multilangaudiosamples.s3.us-east-2.amazonaws.com/audio_samples.zip", DATA_DIR)
! unzip -o $DATA_DIR/audio_samples.zip -d $DATA_DIR
! rm $DATA_DIR/audio_samples.zip

DATA_DIR = os.path.join(DATA_DIR, "audio_samples")

We downloaded audio and text samples in `English` and `Spanish`:

In [ ]:
! ls $DATA_DIR

Data folder for each language contains both audio and text files. Note, the text file and the audio file share the same base name. For example, an audio file `example.wav` should have a corresponding text file called `example.txt`.

In [ ]:
! ls $DATA_DIR/es/audio/ $DATA_DIR/es/text/

# 3. Segmentation of a single file (Spanish sample)

Let's listen to our Spanish audio sample:

In [ ]:
base_name_es = "el19demarzoyel2demayo_03_perezgaldos"
Audio(f"{DATA_DIR}/es/audio/{base_name_es}.wav")

Let's take a look at the ground truth text:

In [ ]:
text = f"{DATA_DIR}/es/text/{base_name_es}.txt"
! cat $text

As one probably noticed, the audio file contains a prologue and an epilogue that are missing in the corresponding text. The segmentation algorithm could handle extra audio fragments at the end and the beginning of the audio, but prolonged untranscribed audio segments in the middle of the file could deteriorate segmentation results. That is why, it is recommended to normalize text, so that transcripts contain spoken equivalents of abbreviations and numbers.

## 3.1. Prepare Text and Audio

We're going to use `prepare_data.py` script to prepare both text and audio data for segmentation.

### Text preprocessing:
* the text will be roughly split into sentences and stored under '$OUTPUT_DIR/processed/*.txt' where each sentence is going to start with a new line (we're going to find alignments for these sentences in the next steps)
* to change the lengths of the final sentences/fragments, specify additional punctuation marks to split the text into fragments, use `--additional_split_symbols` argument. Use `|` as a separator between symbols, for example: `--additional_split_symbols=;|:`
* `max_length` argument - max number of words in a segment for alignment (used only if there are no punctuation marks present in the original text. Long non-speech segments are better for segments split and are more likely to co-occur with punctuation marks. Random text split could deteriorate the quality of the alignment.
* out-of-vocabulary words will be removed based on pre-trained ASR model vocabulary, and the text will be changed to lowercase 
* sentences for alignment with the original punctuation and capitalization will be stored under  `$OUTPUT_DIR/processed/*_with_punct.txt`
* numbers will be converted from written to their spoken form with `num2words` package. For English, it's recommended to use NeMo normalization tool use `--use_nemo_normalization` argument (not supported if running this segmentation tutorial in Colab, see the text normalization tutorial: [`https://github.com/NVIDIA/NeMo-text-processing/blob/main/tutorials/Text_(Inverse)_Normalization.ipynb`](https://colab.research.google.com/github/NVIDIA/NeMo-text-processing/blob/main/tutorials/Text_(Inverse)_Normalization.ipynb) for more details). Even `num2words` normalization is usually enough for proper segmentation. However, it does not take audio into account. NeMo supports audio-based normalization for English, German and Russian languages that can be applied to the segmented data as a post-processing step. Audio-based normalization produces multiple normalization options. For example, `901` could be normalized as `nine zero one` or `nine hundred and one`. The audio-based normalization chooses the best match among the possible normalization options and the transcript based on the character error rate. See [https://github.com/NVIDIA/NeMo-text-processing/blob/main/nemo_text_processing/text_normalization/normalize_with_audio.py](https://github.com/NVIDIA/NeMo-text-processing/blob/main/nemo_text_processing/text_normalization/normalize_with_audio.py) for more details.

### Audio preprocessing:
* non '.wav' audio files will be converted to `.wav` format
* audio files will be resampled to 16kHz (sampling rate used during training NeMo ASR models)
* stereo tracks will be converted to mono
* In some cases, if an audio contains a very long untranscribed prologue, increasing `--cut_prefix` value might help improve segmentation quality.


The `prepare_data.py` will preprocess all `.txt` files found in the `--in_text=$DATA_DIR` and all audio files located at `--audio_dir=$DATA_DIR`.

We are going to use [Spanish Citrinet-512 model](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_es_citrinet_512).

In [ ]:
MODEL = "stt_es_citrinet_512" 
OUTPUT_DIR = WORK_DIR + "/es_output"

! rm -rf $OUTPUT_DIR

! python $TOOLS_DIR/prepare_data.py \
--in_text=$DATA_DIR/es/text \
--output_dir=$OUTPUT_DIR/processed/ \
--language='en' \
--model=$MODEL \
--audio_dir=$DATA_DIR/es/audio

The following four files should be generated and stored at the `$OUTPUT_DIR/processed` folder:

* el19demarzoyel2demayo_03_perezgaldos.txt (lower cased and normalized text with punctuation removed, each line represents an utterance for alignment)
* el19demarzoyel2demayo_03_perezgaldos.wav (.wav mono file, 16kHz)
* el19demarzoyel2demayo_03_perezgaldos_with_punct.txt  (raw utterances for alignment with punctuation and case preserved)
* el19demarzoyel2demayo_03_perezgaldos_with_punct_normalized.txt (normalized utterances for alignment utterance for alignment with punctuation and case preserved)

In [ ]:
! ls $OUTPUT_DIR/processed

The `.txt` file without punctuation contains preprocessed text phrases that we're going to align within the audio file. Here, we split the text into sentences. Each line should contain a text snippet for alignment.

In [ ]:
! head $OUTPUT_DIR/processed/el19demarzoyel2demayo_03_perezgaldos.txt

## 3.2. Run CTC-Segmentation

In this step, we're going to use the [`ctc-segmentation`](https://github.com/lumaku/ctc-segmentation) to find the start and end time stamps for the segments we created during the previous step.


As described in the [CTC-Segmentation of Large Corpora for German End-to-end Speech Recognition](https://arxiv.org/pdf/2007.09127.pdf), the algorithm is relying on a CTC-based ASR model to extract utterance segments with exact time-wise alignments. 

In [ ]:
WINDOW = 8000

! python $TOOLS_DIR/run_ctc_segmentation.py \
--output_dir=$OUTPUT_DIR \
--data=$OUTPUT_DIR/processed \
--model=$MODEL \
--window_len=$WINDOW 

`WINDOW` parameter might need to be adjusted depending on the length of the utterance one wants to align, the default value should work in most cases. By default, if the alignment is not found for the initial `WINDOW` size, the window size will be doubled a few times to re-try backtracing. 

Let's take a look at the generated alignments.

In [ ]:
alignment_file = f"{WINDOW}_{base_name_es}_segments.txt"
! head -n 3 $OUTPUT_DIR/segments/$alignment_file

The expected output for our audio sample looks like this:

```
<PATH_TO>/processed/el19demarzoyel2demayo_03_perezgaldos.wav
11.518331881862931 25.916246734191596 -1.2629864680237006 | entraron en la habitación donde estábamos y al punto que d mauro vio a su sobrina dirigiose a ella con los brazos abiertos y al estrecharla en ellos exclamó endulzando la voz ¡inés de mi alma inocente hija de mi prima juana | Entraron en la habitación donde estábamos, y al punto que D.  Mauro vio a su sobrina dirigiose a ella con los brazos abiertos, y al estrecharla en ellos, exclamó endulzando la voz: -¡Inés de mi alma, inocente hija de mi prima Juana! | Entraron en la habitación donde estábamos, y al punto que D.  Mauro vio a su sobrina dirigiose a ella con los brazos abiertos, y al estrecharla en ellos, exclamó endulzando la voz: - Inés de mi alma, inocente hija de mi prima Juana!
25.756269902499053 28.155922377887165 -0.0003830735786323203 | al fin al fin te veo | Al fin, al fin te veo. | Al fin, al fin te veo.
...
```

**Details of the file content**:
- the first line of the file contains the path to the original audio file
- all subsequent lines contain:
  * the first number is the start of the segment (in seconds)
  * the second one is the end of the segment (in seconds)
  * the third value - alignment confidence score (in log space)
  * text fragments corresponding to the timestamps
  * original text without pre-processing
  * normalized text

Finally, we're going to split the original audio file into segments based on the found alignments. We're going to save only segments with alignment score above the threshold value (default threshold=-2:
* high scored clips (segments with the segmentation score above the threshold value)
* low scored clips (segments with the segmentation score below the threshold)
* deleted segments (segments that were excluded during the alignment. For example, in our sample audio file, the prologue and epilogue that don't have the corresponding transcript were excluded. Oftentimes, deleted files also contain such things as clapping, music, or hard breathing. 

The alignment score values depend on the pre-trained model quality and the dataset.

Also note, that the `OFFSET` parameter is something one might want to experiment with since timestamps have a delay (offset) depending on the model.


In [ ]:
OFFSET = 0
THRESHOLD = -2

! python $TOOLS_DIR/cut_audio.py \
--output_dir=$OUTPUT_DIR \
--alignment=$OUTPUT_DIR/segments/ \
--threshold=$THRESHOLD \
--offset=$OFFSET

## 3.3. Transcribe segmented audio

The transcripts will be saved in a new manifest file in `pred_text` field.

In [ ]:
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/asr/transcribe_speech.py')

! python transcribe_speech.py \
pretrained_name=$MODEL \
dataset_manifest=$OUTPUT_DIR/manifests/manifest.json \
output_filename=$OUTPUT_DIR/manifests/manifest_transcribed.json

In [ ]:
def plot_signal(signal, sample_rate):
    """ Plot the signal in time domain """
    fig_signal = go.Figure(
        go.Scatter(x=np.arange(signal.shape[0])/sample_rate,
                   y=signal, line={'color': 'green'},
                   name='Waveform',
                   hovertemplate='Time: %{x:.2f} s<br>Amplitude: %{y:.2f}<br><extra></extra>'),
        layout={
            'height': 200,
            'xaxis': {'title': 'Time, s'},
            'yaxis': {'title': 'Amplitude'},
            'title': 'Audio Signal',
            'margin': dict(l=0, r=0, t=40, b=0, pad=0),
        }
    )
    fig_signal.show()
    
def display_samples(manifest):
    """ Display audio and reference text."""
    with open(manifest, 'r') as f:
        for line in f:
            sample = json.loads(line)
            sample_rate, signal = wav.read(sample['audio_filepath'])
            plot_signal(signal, sample_rate)
            display(Audio(sample['audio_filepath']))
            display('Reference text:       ' + sample['text_no_preprocessing'])
            if 'pred_text' in sample:
                display('ASR transcript: ' + sample['pred_text'])
            print(f"Score: {sample['score']}")
            print('\n' + '-' * 110)

Let's examine the high scored segments we obtained.

The `Reference text` in the next cell represents the original text without pre-processing, while `ASR transcript` is an ASR model prediction with greedy decoding. Also notice, that `ASR transcript` in some cases contains errors that could decrease the alignment score, but usually it doesn’t hurt the quality of the aligned segments.

Displaying audio in Jupyter Notebook could be slow, it's recommended to use [Speech Data Explorer](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/tools/speech_data_explorer.html) to analyze speech data.

In [ ]:
# let's examine only a few first samples
! head -n 2 $OUTPUT_DIR/manifests/manifest_transcribed.json > $OUTPUT_DIR/manifests/samples.json

display_samples(f"{OUTPUT_DIR}/manifests/samples.json")

# 4. Processing of multiple files (English samples)

Up until now, we were processing only one file at a time, but to create a large dataset processing of multiple files simultaneously could help speed up things considerably. 

Our English data folder contains 2 audio files and corresponding text files:

In [ ]:
! ls $DATA_DIR/en/audio $DATA_DIR/en/text

We are going to use `run_segmentation.sh` to perform all the above steps starting from the text and audio preprocessing to segmentation and manifest creation in a single step:

`run_segmentation.sh` script takes `DATA_DIR` argument and assumes that it contains folders `text` and `audio`.
An example of the `DATA_DIR` folder structure:


--DATA_DIR

     |----audio
            |---1.mp3
            |---2.mp3
            
     |-----text
            |---1.txt
            |---2.txt

`run_segmentation.sh` could use multiple `WINDOW` sizes for segmentation, and then adds segments that were similarly aligned with various window sizes to `verified_segments` folder. This could be useful to reduce the amount of manual work while checking the alignment quality.

In [ ]:
MODEL = "QuartzNet15x5Base-En" # "stt_en_citrinet_512_gamma_0_25" 
OUTPUT_DIR_2 = WORK_DIR + "/en_output"

! rm -rf $OUTPUT_DIR_2

! bash $TOOLS_DIR/../run_segmentation.sh \
--MODEL_NAME_OR_PATH=$MODEL \
--DATA_DIR=$DATA_DIR/en \
--OUTPUT_DIR=$OUTPUT_DIR_2 \
--SCRIPTS_DIR=$TOOLS_DIR \
--MIN_SCORE=$THRESHOLD  \
--USE_NEMO_NORMALIZATION=False

Manifest file with segments with alignment score above the threshold values are saved in `en/output/manifests/manifest.json`.

Next, we are going to run `run_filter.sh`. The script does the following:
* adds ASR transcripts to the manifest
* calculates and saves metrics such as Word Error Rate (WER), Character Error Rate (CER), CER at the tails of the audio file, word difference between reference and transcript, mean absolute values at the tails of the audio.
* filters out samples that do not satisfy threshold values and saves selected segments in `manifest_transcribed_metrics_filtered.json`.

Note, it's better to analyze the manifest with metrics in Speech Data Explorer to decide on what thresholds should be used for final sample selection.

In [ ]:
! bash $TOOLS_DIR/../run_filter.sh \
--SCRIPTS_DIR=$TOOLS_DIR \
--MODEL_NAME_OR_PATH=stt_en_conformer_ctc_large \
--MANIFEST=$OUTPUT_DIR_2/manifests/manifest.json \
--INPUT_AUDIO_DIR=$DATA_DIR/en/audio/

In [ ]:
# let's examine only a few first samples
! head -n 2 $OUTPUT_DIR_2/manifests/manifest_transcribed_metrics_filtered.json > $OUTPUT_DIR_2/manifests/samples.json

display_samples(f"{OUTPUT_DIR_2}/manifests/samples.json")

# Next Steps

- Check out [NeMo Speech Data Explorer tool](https://github.com/NVIDIA/NeMo/tree/main/tools/speech_data_explorer#speech-data-explorer) to interactively evaluate the aligned segments.
- Try Audio-based normalization tool.

# References
Kürzinger, Ludwig, et al. ["CTC-Segmentation of Large Corpora for German End-to-End Speech Recognition."](https://arxiv.org/abs/2007.09127) International Conference on Speech and Computer. Springer, Cham, 2020.